In [ ]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import StopLossCriteria, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 1
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC])
reward_scheme = RiskAdjustedReturns(return_algorithm="sortino")

csv_file = tf.keras.utils.get_file(
    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

# ohlcv_data = pd.read_csv('./data/updateOHLCdata_master.csv')
ohlcv_data = ohlcv_data.iloc[0:10000]

exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN)

wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallet_tuples=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

strategy.run(steps=50000)

W1209 20:40:34.074778 4361977280 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/module.py:31: The name tf.enable_resource_variables is deprecated. Please use tf.compat.v1.enable_resource_variables instead.

W1209 20:40:34.109104 4361977280 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:46: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W1209 20:40:34.109957 4361977280 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:47: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W1209 20:40:34.110806 4361977280 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:48: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train

Observation Data:
['open' 'high' 'low' 'close' 'volume' 'volume usd' 'USD' 'BTC'
 'USD_locked' 'BTC_locked']


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
I1209 20:41:07.031335 4361977280 trading_environment.py:253] Order: OrderStatus.PENDING | TradeSide.BUY | TradeType.MARKET | USD/BTC | 9000.00 USD | 11622.61 | None -> OrderStatus.PENDING | TradeSide.SELL | TradeType.MARKET | 

I1209 20:41:07.328406 4361977280 trading_environment.py:360] P/L: 0.99187265202948
I1209 20:41:07.328999 4361977280 trading_environment.py:361] Reward (10): 0.0854245262035675
I1209 20:41:07.331789 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  9852.441461    9
I1209 20:41:07.333511 4361977280 trading_environment.py:253] Order: None
I1209 20:41:07.360674 4361977280 trading_environment.py:359] Observation: [1.17749700e-04 1.18344100e-04 1.16900000e-04 1.16962800e-04
 8.63300000e-06 1.01545823e-01 1.00000000e-16 8.42356850e-09
 1.00000000e-16 1.00000000e-16]
I1209 20:41:07.361598 4361977280 trading_environment.py:360] P/L: 0.9856223642766799
I1209 20:41:07.362134 4361977280 trading_environment.py:361] Reward (11): 0.16421650894477566
I1209 20:41:07.365466 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked   net_worth step
0  0.842357         

I1209 20:41:07.734512 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  9938.774613   21
I1209 20:41:07.736548 4361977280 trading_environment.py:253] Order: None
I1209 20:41:07.758301 4361977280 trading_environment.py:359] Observation: [1.16700000e-04 1.17987700e-04 1.16667300e-04 1.17987700e-04
 3.84410000e-06 4.51158611e-02 1.00000000e-16 8.42356850e-09
 1.00000000e-16 1.00000000e-16]
I1209 20:41:07.759457 4361977280 trading_environment.py:360] P/L: 0.9829461965959999
I1209 20:41:07.760350 4361977280 trading_environment.py:361] Reward (23): 0.013210277088272304
I1209 20:41:07.763071 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  9830.304323   22
I1209 20:41:07.765655 4361977280 trading_environment.py:253] Order: None
I1209 20:41:07.787279 4361977280 trading_environment.py:359] O

...

I1209 20:41:56.189869 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  9501.692496  823
I1209 20:41:56.191612 4361977280 trading_environment.py:253] Order: None
I1209 20:41:56.222388 4361977280 trading_environment.py:359] Observation: [1.1251150e-04 1.1344470e-04 1.1186090e-04 1.1279890e-04 6.6778000e-06
 7.5242322e-02 1.0000000e-16 8.4235685e-09 1.0000000e-16 1.0000000e-16]
I1209 20:41:56.223722 4361977280 trading_environment.py:360] P/L: 0.94184002516084
I1209 20:41:56.224869 4361977280 trading_environment.py:361] Reward (825): -0.0024276983567056154
I1209 20:41:56.229026 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked   net_worth step
0  0.842357         0.0  0.0         0.0  9477.48316  824
I1209 20:41:56.231337 4361977280 trading_environment.py:253] Order: None
I1209 20:41:56.263617 4361977280 trading_environment.py:359] Observation: 

I1209 20:41:56.648753 4361977280 trading_environment.py:253] Order: None
I1209 20:41:56.677362 4361977280 trading_environment.py:359] Observation: [1.1458390e-04 1.1490280e-04 1.1248020e-04 1.1258180e-04 1.1730400e-05
 1.3345078e-01 1.0000000e-16 8.4235685e-09 1.0000000e-16 1.0000000e-16]
I1209 20:41:56.678949 4361977280 trading_environment.py:360] P/L: 0.9670256523199999
I1209 20:41:56.679677 4361977280 trading_environment.py:361] Reward (837): -0.0010663237443548092
I1209 20:41:56.683052 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  9652.053192  836
I1209 20:41:56.684990 4361977280 trading_environment.py:253] Order: None
I1209 20:41:56.715018 4361977280 trading_environment.py:359] Observation: [1.14800000e-04 1.15328700e-04 1.13269800e-04 1.14583900e-04
 8.33870000e-06 9.54351451e-02 1.00000000e-16 8.42356850e-09
 1.00000000e-16 1.00000000e-16]
I1209 20:41:56.716338 4361977280 t

I1209 20:41:57.132264 4361977280 trading_environment.py:360] P/L: 0.90283132225728
I1209 20:41:57.132944 4361977280 trading_environment.py:361] Reward (849): -0.005847201685961374
I1209 20:41:57.135994 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  9011.533474  848
I1209 20:41:57.138736 4361977280 trading_environment.py:253] Order: None
I1209 20:41:57.169173 4361977280 trading_environment.py:359] Observation: [1.0717920e-04 1.0730000e-04 1.0575090e-04 1.0698000e-04 1.3205700e-05
 1.4067293e-01 1.0000000e-16 8.4235685e-09 1.0000000e-16 1.0000000e-16]
I1209 20:41:57.170340 4361977280 trading_environment.py:360] P/L: 0.88358683789064
I1209 20:41:57.171262 4361977280 trading_environment.py:361] Reward (850): -0.005716323542145998
I1209 20:41:57.174093 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  

I1209 20:41:57.590163 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  8226.633794  860
I1209 20:41:57.591885 4361977280 trading_environment.py:253] Order: None
I1209 20:41:57.623856 4361977280 trading_environment.py:359] Observation: [9.7835400e-05 9.8730800e-05 9.7110300e-05 9.7662100e-05 1.3822700e-05
 1.3537154e-01 1.0000000e-16 8.4235685e-09 1.0000000e-16 1.0000000e-16]
I1209 20:41:57.624763 4361977280 trading_environment.py:360] P/L: 0.8342146187845599
I1209 20:41:57.625383 4361977280 trading_environment.py:361] Reward (862): -0.011120739163575932
I1209 20:41:57.629199 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  8241.231838  861
I1209 20:41:57.631583 4361977280 trading_environment.py:253] Order: None
I1209 20:41:57.663976 4361977280 trading_environment.py:359] Observatio

I1209 20:41:58.050025 4361977280 trading_environment.py:253] Order: None
I1209 20:41:58.080293 4361977280 trading_environment.py:359] Observation: [1.02452300e-04 1.03188800e-04 1.01540900e-04 1.02673600e-04
 1.15306000e-05 1.18199363e-01 1.00000000e-16 8.42356850e-09
 1.00000000e-16 1.00000000e-16]
I1209 20:41:58.081079 4361977280 trading_environment.py:360] P/L: 0.86583922162868
I1209 20:41:58.081814 4361977280 trading_environment.py:361] Reward (874): -0.008234450655582176
I1209 20:41:58.084879 4361977280 trading_environment.py:362] Performance:         BTC  BTC_locked  USD  USD_locked    net_worth step
0  0.842357         0.0  0.0         0.0  8630.139568  873
I1209 20:41:58.086560 4361977280 trading_environment.py:253] Order: None
I1209 20:41:58.115322 4361977280 trading_environment.py:359] Observation: [1.02787700e-04 1.03680000e-04 1.01405100e-04 1.02452300e-04
 1.60238000e-05 1.63852747e-01 1.00000000e-16 8.42356850e-09
 1.00000000e-16 1.00000000e-16]
I1209 20:41:58.117146 4361